In [2]:
import time
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import logging
import random

In [21]:
total_follow_count = 0
total_unfollow_count = 0
logger = logging.getLogger('twitter_logger')
logger.setLevel(logging.DEBUG)

In [1]:
username = ""
password = ""

In [43]:
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_argument("--auto-open-devtools-for-tabs")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path='chromedriver4')

In [6]:
def twitter_login(username, password):
    if type(username) is not str or type(password) is not str:
        return
    url = "https://twitter.com/i/flow/login"
    driver.get(url)
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id ="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div/div[5]/label/div/div[2]/div/input').send_keys(username)
    driver.find_element_by_xpath('//*[@id ="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div/div[6]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id ="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div/div/div/div/div[3]/div/label/div/div[2]/div/input').send_keys(password)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id ="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div[2]/div/div/div/div/div/div[1]').click()

In [44]:
twitter_login(username, password)

In [8]:
def retrieve_followers(username):
    url = "https://twitter.com/" + username + "/followers/"
    driver.get(url)
    SCROLL_PAUSE_TIME = 1.5
    last_height = driver.execute_script("return document.body.scrollHeight")
    followers_list = []
    while True:
        # Scroll down to bottom
        for a in driver.find_elements_by_xpath('//div[@aria-label="Timeline: Followers"]//a[@role="link"]'):
            url = a.get_property('href')
            url = url.replace("https://twitter.com/", "@")
            if 'search' in url:
                continue
            if 't.co' in url:
                continue
            if 'https' in url:
                continue
            if url not in followers_list:
                followers_list.append(url)
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return followers_list

In [12]:
# Delete Duplicates in List
to_follow = list(set([x.lower() for x in to_follow]))

In [13]:
def retrieve_following(username):
    url = "https://twitter.com/" + username + "/following/"
    driver.get(url)
    SCROLL_PAUSE_TIME = 1.5
    last_height = driver.execute_script("return document.body.scrollHeight")
    following_list = []
    while True:
        for a in driver.find_elements_by_xpath('//div[@aria-label="Timeline: Following"]//a[@role="link"]'):
            url = a.get_property('href')
            url = url.replace("https://twitter.com/", "@")
            if 'search' in url:
                continue
            if 't.co' in url:
                continue
            if 'https' in url:
                continue
            if url not in following_list:
                following_list.append(url)
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return following_list

In [14]:
def unfollow_list(username):
    unfollowing_list = []
    followers = retrieve_followers(username)
    following = retrieve_following(username)
    for x in following:
        if x not in followers:
            unfollowing_list.append(x)
    return unfollowing_list

In [15]:
def get_user_information(username):
    users_info = pd.DataFrame(columns = ["Username", "Followers", "Following", "Join Date", "Work", "Location", "Website", "Bio", "Birthday"])
    try:
        website = driver.find_element_by_xpath('//div[contains(@data-testid,"UserProfileHeader_Items")]//a[1]').get_attribute("href")
    except Exception as e:
        website = ""
    try:
        bio = driver.find_element_by_xpath('//div[@data-testid="UserDescription"]').text
    except Exception as e:
        bio = ""
    try:
        work = driver.find_element_by_xpath('//span[contains(@data-testid, "Professional")]').text
    except:
        work = ""
    try:
        location = driver.find_element_by_xpath('//span[contains(@data-testid, "Location")]').text
    except:
        location = ""
    try:
        birthday = driver.find_element_by_xpath('//span[contains(@data-testid, "Birthdate")]').text[5:]
    except:
        birthday = ""
    try:
        join_date = driver.find_element_by_xpath('//span[contains(@data-testid, "JoinDate")]').text[7:]
    except:
        join_date = ''
    try:
        followers = driver.find_element_by_xpath(
            '//a[contains(@href,"/followers")]/span[1]/span[1]').text.replace(",", "")
        following = driver.find_element_by_xpath(
            '//a[contains(@href,"/following")]/span[1]/span[1]').text.replace(",", "")
        if 'K' in followers:
            followers = int(float(followers[:-1]) * 1000)
        elif 'M' in followers:
            followers = int(float(followers[:-1]) * 1000000)
        else:
            followers = int(followers)
        if 'K' in following:
            following = int(float(following[:-1]) * 1000)
        elif 'M' in following:
            following = int(float(following[:-1] * 1000000))
        else:
            following = int(following)
    except Exception as e:
        return
    users_info = [username, followers, following, join_date, work, location, website, bio, birthday]
    return users_info

In [71]:
def follow_from_list(accounts):
    unfollowers_list = pd.read_csv('~/Downloads/twitter-data/unfollowing-list.csv')
    for account in accounts:
        driver.get("https://twitter.com/" + account)
        if account in unfollowers_list['Username'].unique() or '@' + account in unfollowers_list['Username'].unique():
            logger.info(f"Account {account} is in the unfollowers list!")
            continue
        try:
            x = driver.find_element_by_xpath('//div[contains(@data-testid,"FollowIndicator")]').text
            if x == 'Follows you':
                logger.info(f'Account {account} already follows you')
                continue
        except:
            try:
                driver.get('https://twitter.com/' + account)
                time.sleep(5)
                # [username, followers, following, join_date, work, location, website, bio, birthday] = 
                info = get_user_information(account)
                followers = info[1]
                following = info[2]
                join_date = info[3]
                work = info[4]
                location = info[5]
                website = info[6]
                bio = info[7]
                birthday = info[8]
                elem = driver.find_elements(By.CSS_SELECTOR, '.r-yfoy6g > .r-jwli3a')
                global total_follow_count
                if followers > 10000:
                    print(f'Account {account} is too popular to follow back')
                    continue
                elif following < 250 and followers > 500:
                    print(f'Account {account} is unlikely to follow back')
                    continue
                elif followers > (following * 2):
                    print(f'Account {account} is too picky to follow back')
                    continue
                elif len(elem) > 0 and (elem[0].text == 'This account doesn’t exist'):
                    print(f'Account {account}does not exist.')
                    continue
                else: 
                    driver.find_element(By.CSS_SELECTOR, '.css-1dbjc4n:nth-child(1) > .css-18t94o4:nth-child(1) > .css-901oao > .css-901oao > .css-901oao').click()
                    time.sleep(10)
                    print(f'Successful subscribed to {account}.')
                    total_follow_count =  total_follow_count + 1
                    if(total_follow_count > 300):
                        print("Total Follow Count REACHED!")
                        break
                    try: 
                        elem_1 = driver.find_element_by_xpath("//div[@data-testid='confirmationSheetConfirm']//span[contains(text(),'Unfollow')]")
                        if 'Unfollow' in elem_1.text:
                            print(f'Account {account}is already followed')
                            total_follow_count =  total_follow_count - 1
                            continue
                    except:
                        continue
            except:   
                print(f'Subscribtion to {account} not possible.') 

In [ ]:
# Create an unfollowing list 
unfollowers_list = pd.DataFrame()
# Set the save path for the unfollowing list here
PATH = ""
unfollowers_list.to_csv(PATH)

In [915]:
def unfollow_from_list(unfollowing_list):
    # Create an Unfollowing List First
    global PATH
    unfollowers_list = pd.read_csv(PATH)
    users_info = pd.DataFrame(columns = ["Username", "Followers", "Following", "Join Date", "Work", "Location", "Website", "Bio", "Birthday"])
    for account in unfollowing_list:
        try:
            driver.get('https://twitter.com/' + account)
            time.sleep(20)
            elem = driver.find_elements(By.CSS_SELECTOR, '.r-yfoy6g > .r-jwli3a')
            if len(elem) > 0 and (elem[0].text == 'This account doesn’t exist'):
                logger.info(f'Account {account} does not exist.')
                continue      
            try:
                x = driver.find_element_by_xpath('//div[contains(@data-testid,"FollowIndicator")]').text
                logger.info(f'Account {account} already follows you')
                if x == 'Follows you':
                    continue
            except: 
                # elem_1 = driver.find_elements(By.CSS_SELECTOR, '.r-poiln3 > .r-bcqeeo > .r-qvutc0')
                if driver.find_element(By.CSS_SELECTOR, '.css-1dbjc4n:nth-child(1) > .css-18t94o4:nth-child(1) > .css-901oao > .css-901oao > .css-901oao').text == 'Follow':
                    logger.info(f'Account {account} is already not followed.')
                    continue
                    time.sleep(2)
                else:
                    users_info.loc[len(users_info)] = get_user_information(account)
                    driver.find_element(By.CSS_SELECTOR, '.css-1dbjc4n:nth-child(1) > .css-18t94o4:nth-child(1) > .css-901oao > .css-901oao > .css-901oao').click()
                    time.sleep(10)
                    driver.find_element_by_xpath("//div[@data-testid='confirmationSheetConfirm']//span[contains(text(),'Unfollow')]").click()
                    global total_unfollow_count
                    total_unfollow_count =  total_unfollow_count + 1
                    time.sleep(2)
        except:   
            logger.info('Exception') 
    unfollowers_list = pd.concat([unfollowers_list, users_info], ignore_index = True)
    unfollowers_list.to_csv("~/Downloads/twitter-data/unfollowing-list.csv", index = False)